# Support Vector Machine - Test Features and Error Metrics
This notebook tests features, error metrics and ideas from 'Evaluating Segmentation Error Without Ground Truth' by Kohlberger et al.

author = Caroline Magg <br>
date = 29 July 2020 <br> 

______________________________________
history: <br>
2020-29-07 Dice, Volume Overlap, Hausdorff <br>
2020-30-07 Jaccard <br>
2020-18-08 Average surface error and modified Hausdorff <br>
2020-27-08 Testing SVM dependent features <br> <br>


In [ ]:
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import cv2
import scipy
import logging as log
import skimage.segmentation as segmentation
from scipy.spatial.distance import cdist

In [ ]:
log.basicConfig(format='%(levelname)s:%(message)s', level=log.INFO)

### Add dependencies

In [ ]:
# add KidsBrainProject main folder to paths
sys.path.append(os.path.abspath('../../'))
sys.path.append(os.path.abspath('../utils/'))

In [ ]:
# add path to data here
path_data = "../../Data/" 

data_brain = pd.read_csv("../../Data/data_Brain.csv")

# Segmentation Error Metrics

## Synthetic Test Data

In [ ]:
groundtruth = np.zeros((100,100))
cv2.rectangle(groundtruth, (0,0), (49,49), 255, -1)
plt.imshow(groundtruth), np.sum(groundtruth==255)

In [ ]:
# smaller prediction
predicted1 = np.zeros((100,100))
cv2.rectangle(predicted1, (0,0), (39,39), 255, -1)
plt.imshow(predicted1), np.sum(predicted1==255)

In [ ]:
# bigger prediction
predicted2 = np.zeros((100,100))
cv2.rectangle(predicted2, (0,0), (57,57), 255, -1)
plt.imshow(predicted2), np.sum(predicted2==255)

In [ ]:
# no overlap
predicted3 = np.zeros((100,100))
cv2.rectangle(predicted3, (50,50), (99,99), 255, -1)
plt.imshow(predicted3), np.sum(predicted2==255)

In [ ]:
plt.imshow(groundtruth+predicted1/255*200)
plt.show()
plt.imshow(groundtruth+predicted2/255*200)
plt.show()
plt.imshow(groundtruth+predicted3/255*200)
plt.show()

## Dice Coefficient
1 - perfect segmentation, 0 - completely failed

In [ ]:
def dice_coeff(gt, pred, k=255):
    return np.sum(pred[gt==k])*2.0 / (np.sum(pred) + np.sum(gt))

In [ ]:
dice_coeff(groundtruth, predicted1)

In [ ]:
dice_coeff(groundtruth, predicted2)

In [ ]:
dice_coeff(groundtruth, groundtruth)

In [ ]:
dice_coeff(groundtruth, predicted3)

## Jaccard Distance

In [ ]:
def jaccard_coeff(gt, pred, k=255):
    return np.sum(pred[gt==k]) / (np.sum(pred) + np.sum(gt) - np.sum(pred[gt==k]))

In [ ]:
jaccard_coeff(groundtruth, predicted1)

In [ ]:
jaccard_coeff(groundtruth, predicted2)

In [ ]:
jaccard_coeff(groundtruth, groundtruth)

In [ ]:
jaccard_coeff(groundtruth, predicted3)

## Volumentric Overlap Error
0% - perfect segmentation (pred==gt), 100% - no overlap at all

In [ ]:
def vol_overlap_error(gt, pred, k=255):
    return 1 - np.sum(pred[gt==k]) / (np.sum(pred) + np.sum(gt) - np.sum(pred[gt==k]))

In [ ]:
vol_overlap_error(groundtruth, predicted1)

In [ ]:
vol_overlap_error(groundtruth, predicted2)

In [ ]:
vol_overlap_error(groundtruth, groundtruth)

In [ ]:
vol_overlap_error(groundtruth, predicted3)

## Mask -> Surface

In [ ]:
im3, contours, hierarchy = cv2.findContours(predicted1.astype(np.uint8),cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)

In [ ]:
plt.imshow(cv2.drawContours(np.zeros_like(im3), contours, -1, 255, 1))

## Hausdorff Distance
symmetric surface-to-surface metric <br>
the bigger, the more distinct are the surfaces, 0 - perfect overlap <br>
due to max min, it makes no difference if the contour or the mask is used <br>
for the modified version, it makes a difference, due to the mean <br>

In [ ]:
def hausdorff_distance(gt, pred):
    gt3, contours, hierarchy = cv2.findContours(gt.astype(np.uint8),cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
    gt_contour = cv2.drawContours(np.zeros_like(gt3), contours, -1, 255, 1)
    pred3, contours, hierarchy = cv2.findContours(pred.astype(np.uint8),cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
    pred_contour = cv2.drawContours(np.zeros_like(pred3), contours, -1, 255, 1)    
    distance = cdist(gt_contour, pred_contour, 'euclidean')
    dist1 = np.max(np.min(distance, axis=0))
    dist2 = np.max(np.min(distance, axis=1))
    return max(dist1, dist2)

In [ ]:
hausdorff_distance(groundtruth, predicted1)

In [ ]:
hausdorff_distance(groundtruth, predicted2)

In [ ]:
hausdorff_distance(groundtruth, groundtruth)

In [ ]:
hausdorff_distance(groundtruth, predicted3)

In [ ]:
def hausdorff_distance(gt, pred):   
    distance = cdist(gt, pred, 'euclidean')
    dist1 = np.max(np.min(distance, axis=0))
    dist2 = np.max(np.min(distance, axis=1))
    return max(dist1, dist2)

In [ ]:
hausdorff_distance(groundtruth, predicted1)

In [ ]:
hausdorff_distance(groundtruth, predicted2)

In [ ]:
hausdorff_distance(groundtruth, groundtruth)

In [ ]:
hausdorff_distance(groundtruth, predicted3)

In [ ]:
def mod_hausdorff_distance(gt, pred):
    gt3, contours, hierarchy = cv2.findContours(gt.astype(np.uint8),cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
    gt_contour = cv2.drawContours(np.zeros_like(gt3), contours, -1, 255, 1)
    pred3, contours, hierarchy = cv2.findContours(pred.astype(np.uint8),cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
    pred_contour = cv2.drawContours(np.zeros_like(pred3), contours, -1, 255, 1)    
    distance = cdist(gt_contour, pred_contour, 'euclidean')
    dist1 = np.mean(np.min(distance, axis=0))
    dist2 = np.mean(np.min(distance, axis=1))
    return max(dist1, dist2)

In [ ]:
mod_hausdorff_distance(groundtruth, predicted1)

In [ ]:
mod_hausdorff_distance(groundtruth, predicted2)

In [ ]:
mod_hausdorff_distance(groundtruth, groundtruth)

In [ ]:
mod_hausdorff_distance(groundtruth, predicted3)

In [ ]:
def mod_hausdorff_distance(gt, pred):
    distance = cdist(gt, pred, 'euclidean')
    dist1 = np.mean(np.min(distance, axis=0))
    dist2 = np.mean(np.min(distance, axis=1))
    return max(dist1, dist2)

In [ ]:
mod_hausdorff_distance(groundtruth, predicted1)

In [ ]:
mod_hausdorff_distance(groundtruth, predicted2)

In [ ]:
mod_hausdorff_distance(groundtruth, groundtruth)

In [ ]:
mod_hausdorff_distance(groundtruth, predicted3)

using the scipy implementation of directed_hausdorff

In [ ]:
from scipy.spatial.distance import directed_hausdorff

In [ ]:
directed_hausdorff(groundtruth, predicted1)[0]

In [ ]:
max(directed_hausdorff(groundtruth, predicted1)[0], directed_hausdorff(predicted1, groundtruth)[0])

## Average surface error
mean of minimum per-vertex surface distances

In [ ]:
def average_surface_error(gt, pred):
    gt3, contours, hierarchy = cv2.findContours(gt.astype(np.uint8),cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
    gt_contour = cv2.drawContours(np.zeros_like(gt3), contours, -1, 255, 1)
    pred3, contours, hierarchy = cv2.findContours(pred.astype(np.uint8),cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
    pred_contour = cv2.drawContours(np.zeros_like(pred3), contours, -1, 255, 1)    
    distance = cdist(gt_contour, pred_contour, 'euclidean')
    dist1 = np.sum(np.min(distance, axis=0))/np.sum(pred_contour==255) # first term
    dist2 = np.sum(np.min(distance, axis=1))/np.sum(gt_contour==255) # second term
    return 1/2*(dist1+dist2)

In [ ]:
average_surface_error(groundtruth, predicted1)

In [ ]:
average_surface_error(groundtruth, predicted2)

In [ ]:
average_surface_error(groundtruth, groundtruth)

In [ ]:
average_surface_error(groundtruth, predicted3)

In [ ]:
def average_surface_error(gt, pred, k=255):
    distance = cdist(gt, pred, 'euclidean')
    dist1 = np.sum(np.min(distance, axis=0))/np.sum(pred==k) # first term
    dist2 = np.sum(np.min(distance, axis=1))/np.sum(gt==k) # second term
    return 1/2*(dist1+dist2)

In [ ]:
average_surface_error(groundtruth, predicted1)

In [ ]:
average_surface_error(groundtruth, predicted2)

In [ ]:
average_surface_error(groundtruth, groundtruth)

In [ ]:
average_surface_error(groundtruth, predicted3)

## Test SVM Features

In [ ]:
from SVMFeatures import SVMFeatures

In [ ]:
features = SVMFeatures([predicted2], [groundtruth], [predicted1], k=255)
features.segm_value

In [ ]:
features.calculate_dependent_features()

In [ ]:
features.dice_coeff([groundtruth], [predicted1], k=255)

In [ ]:
features.jaccard_distance([groundtruth], [predicted1], k=255)

In [ ]:
features.hausdorff_distance([groundtruth], [predicted1])

In [ ]:
features.mod_hausdorff_distance([groundtruth], [predicted1], k=255)

In [ ]:
features.average_surface_error([groundtruth], [predicted1], k=255)